# RESOLVED

## Objective:

This is trying to determine the cause of the deep field calibration issues that have been occuring.

## Method:

A much more liberal cut will be applied, and classification rates will be calculated. If these are way off, something got lost in translation and there is simply a bug. If not, the color will slowly be reigned in until it matches the isochrone, and I will look for when the classification rate drops off quickly. Wide field objects read in should be saved with their positions, class, r/g band magnitudes. Class should be saved so that I can see if it happens to be some issue with class cutoff, although this wouldn't really make sense.

## Desired Outcome:

Find out what is causing such the huge issues in deep field calibrations within the isochrone pipeline.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import fitsio
import astropy.io.fits as fits
from astropy.table import Table
import healpy as hp
import healsparse as hsp
import skyproj
from os import listdir
import astropy.units as u
from astropy.coordinates import SkyCoord
import Config
import IsochroneConfig as isoConfig
from CropSurveyProperties import *
from GetObjects import *
from TrainAndFullMap import *

Reading fracdet map /hdfs/bechtol/balrog/y3/y3a2_survey_conditions_maps/fracdet/y3a2_griz_o.4096_t.32768_coverfoot_EQU.fits.gz ...
NSIDE = 4096
ORDERING = NESTED in fits file
INDXSCHM = EXPLICIT
Ordering converted to RING


In [2]:
res = isoConfig.res
perCovered = isoConfig.perCovered
numMagBins = isoConfig.numMagBins
sigma = isoConfig.sigma
perMap = isoConfig.perMap
perVar = isoConfig.perVar
numBins = isoConfig.numBins
Class_Cutoff = isoConfig.classCutoff
goldCols = isoConfig.goldCols
qualityCuts = isoConfig.qualityCuts

In [3]:
deepFiles = isoConfig.deepFiles

goldObjectsDir = isoConfig.goldObjectsDir
goldObjectsFiles = isoConfig.goldObjectsFiles

objPosDir = isoConfig.caliDir[:-1] + 'Tests/'

pixFile = isoConfig.pixFile

matStarFile = isoConfig.matStarFile

galaProbFiles = isoConfig.galaProbFiles

starProbFiles = isoConfig.starProbFiles

In [4]:
starPosColorMagFiles = []
galaPosColorMagFiles = []
for i in np.arange(numMagBins):
    starPosColorMagFiles.append(objPosDir + 'StarPosColorMag_Bin' + str(i+1) + '.fits')
    galaPosColorMagFiles.append(objPosDir + 'GalaPosColorMag_Bin' + str(i+1) + '.fits')

In [5]:
def findMatches(angleCutoff, RASource, DECSource, RAMatchCatalog, DECMatchCatalog, nthneighbor=1):
    c = SkyCoord(ra=RASource*u.degree, dec=DECSource*u.degree)
    catalog = SkyCoord(ra=RAMatchCatalog*u.degree, dec=DECMatchCatalog*u.degree)
    idx, d2d, d3d = c.match_to_catalog_sky(catalog, nthneighbor=nthneighbor)
    matches = d2d < angleCutoff
    return matches, d2d

In [6]:
deepCols = isoConfig.deepCols

deepRA = np.array([])
deepDEC = np.array([])
deepClass = np.array([])
deepFlag = np.array([])
deepFlagNir = np.array([])

for deepFile in deepFiles:
    deepData = fitsio.read(deepFile, columns = deepCols)

    deepRA = np.append(deepRA, deepData['RA'])
    deepDEC = np.append(deepDEC, deepData['DEC'])
    deepClass = np.append(deepClass, deepData['KNN_CLASS'])
    deepFlag = np.append(deepFlag, deepData['MASK_FLAGS'])
    deepFlagNir = np.append(deepFlagNir, deepData['MASK_FLAGS_NIR'])

deepFlagCuts = np.where((deepFlag == 0) &
                        (deepFlagNir == 0) &
                        (deepRA < 120) &
                        (deepClass > 0) &
                        (deepClass <= 3))[0]

deepRA = deepRA[deepFlagCuts]
deepDEC = deepDEC[deepFlagCuts]
deepClass = deepClass[deepFlagCuts]

if len(np.where(deepClass == 3)[0]) != 0:
    print('WARNING: Objects with no class are present in this deep field selection. ' + str(len(np.where(deepClass == 3)[0])) + ' object(s) out of ' + str(len(deepClass)) + ' have an ambiguous classification.')

deepPix = np.unique(hp.ang2pix(res, deepRA, deepDEC, lonlat = True, nest = True))

deepPixCheck = np.full(12*(res**2), False, dtype = bool)
deepPixCheck[deepPix] = True

In [7]:
validPix = fitsio.read(pixFile)['PIXEL']
pixCheck = np.full(12*(res**2), False, dtype = bool)
pixCheck[validPix] = True

In [8]:
matStarRMAG = np.sort(fitsio.read(matStarFile)['RMAG'])

splitMags = np.array_split(matStarRMAG, numMagBins)

magBins = [] # This will define the boundary of bins so that the number of objects is constant.
for i in np.arange(len(splitMags) - 1):
    magBins.append(splitMags[i][-1])

In [9]:
for i in np.arange(numMagBins):
    
    starPosFile = starPosColorMagFiles[i]
    galaPosFile = galaPosColorMagFiles[i]
    
    StarRA = []
    StarDEC = []
    StarGMAG = []
    StarRMAG = []

    GalaRA = []
    GalaDEC = []
    GalaGMAG = []
    GalaRMAG = []

    for file in goldObjectsFiles:
        obsData = fitsio.read(goldObjectsDir + file, columns = goldCols)
        FOREGROUND = obsData[goldCols[0]]
        BADREGIONS = obsData[goldCols[1]]
        FOOTPRINT = obsData[goldCols[2]]
        CLASS = obsData[goldCols[3]]
        GMAG = obsData[goldCols[4]]
        RMAG = obsData[goldCols[5]]
        IMAG = obsData[goldCols[6]]
        GMAG_GALA = obsData[goldCols[7]]
        RMAG_GALA = obsData[goldCols[8]]
        IMAG_GALA = obsData[goldCols[9]]
        RA = obsData[goldCols[10]]
        DEC = obsData[goldCols[11]]
        PIX = hp.ang2pix(res, RA, DEC, lonlat = True, nest = True)
        
        # General Quality Cuts
        qualityCut = np.where((FOREGROUND == 0) &
                              (BADREGIONS < 2) &
                              (FOOTPRINT == 1) &
                              (deepPixCheck[PIX]) &
                              (pixCheck[PIX]))[0]
        
        CLASS = CLASS[qualityCut]
        GMAG = GMAG[qualityCut]
        RMAG = RMAG[qualityCut]
        IMAG = IMAG[qualityCut]
        GMAG_GALA = GMAG_GALA[qualityCut]
        RMAG_GALA = RMAG_GALA[qualityCut]
        IMAG_GALA = IMAG_GALA[qualityCut]
        RA = RA[qualityCut]
        DEC = DEC[qualityCut]
        PIX = PIX[qualityCut]
        
        # This defines magnitude cuts in accordance with the magnitude bins.
        if i == 0:
            maxRMAG = magBins[i]
            magCut = np.where(RMAG <= maxRMAG)[0]
            magCutGala = np.where(RMAG_GALA <= maxRMAG)[0]
        elif i == len(magBins):
            minRMAG = magBins[i - 1]
            magCut = np.where(RMAG > minRMAG)[0]
            magCutGala = np.where(RMAG_GALA > minRMAG)[0]
        else:
            minRMAG = magBins[i - 1]
            maxRMAG = magBins[i]
            magCut = np.where((RMAG <= maxRMAG) & (RMAG > minRMAG))[0]
            magCutGala = np.where((RMAG_GALA <= maxRMAG) & (RMAG_GALA > minRMAG))[0]
        
        boolMagCut = np.zeros(len(RMAG))
        boolMagCut[magCut] = 1
        
        boolMagCutGala = np.zeros(len(RMAG_GALA))
        boolMagCutGala[magCutGala] = 1

        # Observed Star Objects

        blueStarCut = np.where((CLASS <= Class_Cutoff) & 
                          (CLASS >= 0) &
                          (boolMagCut > 0) &
                          (RMAG < qualityCuts[0]) &
                          (IMAG < qualityCuts[1]) &
                          (GMAG - RMAG >= -0.3) & 
                          (GMAG - RMAG <= 1))[0]

        STARRA = RA[blueStarCut]
        STARDEC = DEC[blueStarCut]
        STARGMAG = GMAG[blueStarCut]
        STARRMAG = RMAG[blueStarCut]
        
#         isoStarCut = iso.cut_separation('g', 'r', STARGMAG, STARRMAG, np.zeros_like(STARGMAG), np.zeros_like(STARRMAG), radius=0.1)

#         STARRA = STARRA[isoStarCut]
#         STARDEC = STARDEC[isoStarCut]
        
        StarRA.extend(STARRA)
        StarDEC.extend(STARDEC)
        StarGMAG.extend(STARGMAG)
        StarRMAG.extend(STARRMAG)

        # Observed Galaxy Objects

        blueGalaCut = np.where((CLASS <= 3) & 
                          (CLASS >= Class_Cutoff) &
                          (boolMagCutGala > 0) &
                          (RMAG_GALA < qualityCuts[0]) &
                          (IMAG_GALA < qualityCuts[1]) &
                          (GMAG_GALA - RMAG_GALA >= -0.3) &
                          (GMAG_GALA - RMAG_GALA <= 1))[0]

        GALARA = RA[blueGalaCut]
        GALADEC = DEC[blueGalaCut]
        GALAGMAG = GMAG[blueGalaCut]
        GALARMAG = RMAG[blueGalaCut]
        
#         isoGalaCut = iso.cut_separation('g', 'r', GALAGMAG, GALARMAG, np.zeros_like(GALAGMAG), np.zeros_like(GALARMAG), radius=0.1)

#         GALARA = GALARA[isoGalaCut]
#         GALADEC = GALADEC[isoGalaCut]

        GalaRA.extend(GALARA)
        GalaDEC.extend(GALADEC)
        GalaGMAG.extend(GALAGMAG)
        GalaRMAG.extend(GALARMAG)

    StarRA = np.array(StarRA, dtype = object)
    StarDEC = np.array(StarDEC, dtype = object)
    StarGMAG = np.array(StarGMAG, dtype = object)
    StarRMAG = np.array(StarRMAG, dtype = object)

    GalaRA = np.array(GalaRA, dtype = object)
    GalaDEC = np.array(GalaDEC, dtype = object)
    GalaGMAG = np.array(GalaGMAG, dtype = object)
    GalaRMAG = np.array(GalaRMAG, dtype = object)
    
    my_table = Table()
    my_table['RA'] = StarRA.astype(float)
    my_table['DEC'] = StarDEC.astype(float)
    my_table['GMAG'] = StarGMAG.astype(float)
    my_table['RMAG'] = StarRMAG.astype(float)
    my_table.write(starPosFile, overwrite = True)
    
    my_table = Table()
    my_table['RA'] = GalaRA.astype(float)
    my_table['DEC'] = GalaDEC.astype(float)
    my_table['GMAG'] = GalaGMAG.astype(float)
    my_table['RMAG'] = GalaRMAG.astype(float)
    my_table.write(galaPosFile, overwrite = True)

In [10]:
starAdjustments = []
galaAdjustments = []

for i in np.arange(len(starPosColorMagFiles)):
    allStarData = fitsio.read(starPosColorMagFiles[i])
    allStarRA = allStarData['RA']
    allStarDEC = allStarData['DEC']
    print(len(allStarRA))

    allGalaData = fitsio.read(galaPosColorMagFiles[i])
    allGalaRA = allGalaData['RA']
    allGalaDEC = allGalaData['DEC']
    print(len(allGalaRA))
    
    deepStarMatches, _ = findMatches(0.5*u.arcsec, deepRA, deepDEC, allStarRA, allStarDEC)
    deepGalaMatches, _ = findMatches(0.5*u.arcsec, deepRA, deepDEC, allGalaRA, allGalaDEC)

    matchedDeepStarRA = deepRA[deepStarMatches]
    matchedDeepStarDEC = deepDEC[deepStarMatches]
    matchedDeepStarClass = deepClass[deepStarMatches]

    matchedDeepGalaRA = deepRA[deepGalaMatches]
    matchedDeepGalaDEC = deepDEC[deepGalaMatches]
    matchedDeepGalaClass = deepClass[deepGalaMatches]
    
    TSPIX = hp.ang2pix(res, matchedDeepStarRA[np.where(matchedDeepStarClass == 2)[0]], matchedDeepStarDEC[np.where(matchedDeepStarClass == 2)[0]], lonlat = True, nest = True)
    FSPIX = hp.ang2pix(res, matchedDeepStarRA[np.where(matchedDeepStarClass == 1)[0]], matchedDeepStarDEC[np.where(matchedDeepStarClass == 1)[0]], lonlat = True, nest = True)

    TGPIX = hp.ang2pix(res, matchedDeepGalaRA[np.where(matchedDeepGalaClass == 1)[0]], matchedDeepGalaDEC[np.where(matchedDeepGalaClass == 1)[0]], lonlat = True, nest = True)
    FGPIX = hp.ang2pix(res, matchedDeepGalaRA[np.where(matchedDeepGalaClass == 2)[0]], matchedDeepGalaDEC[np.where(matchedDeepGalaClass == 2)[0]], lonlat = True, nest = True)
    
    print(len(TSPIX))
    print(len(FGPIX))
    print(len(TGPIX))
    print(len(FSPIX))
    print(' ')
    
    starCorrProb = np.clip(fitsio.read(starProbFiles[i])['SIGNAL'], 0, 1)
    fullStarProb = np.full(12*(res**2), hp.UNSEEN)
    fullStarProb[validPix] = starCorrProb

    galaCorrProb = np.clip(fitsio.read(galaProbFiles[i])['SIGNAL'], 0, 1)
    fullGalaProb = np.full(12*(res**2), hp.UNSEEN)
    fullGalaProb[validPix] = galaCorrProb
    
    starAdjustments.append(len(TSPIX) / (np.sum(fullStarProb[TSPIX[np.where(pixCheck[TSPIX])[0]]]) + np.sum(fullStarProb[FGPIX[np.where(pixCheck[FGPIX])[0]]])))   
    galaAdjustments.append(len(TGPIX) / (np.sum(fullGalaProb[TGPIX[np.where(pixCheck[TGPIX])[0]]]) + np.sum(fullGalaProb[FSPIX[np.where(pixCheck[FSPIX])[0]]])))
    
# caliTable = Table()
# caliTable['STAR'] = starAdjustments
# caliTable['GALA'] = galaAdjustments
# caliTable.write(calibrationFile, overwrite = True)

2474
4043
917
15
1785
77
 
2471
9748
1161
24
5310
227
 
3039
25292
1340
69
13687
405
 
5843
90944
1681
337
50041
1760
 
41078
118647
1137
446
66352
22591
 
